# Piru Music Composer

In [9]:
import guitarpro
import math

## Loading data

Since GuitarPro format express the music in terms of fret on chords, we should first derive the notes from them


In [10]:
with open('data/Guitar Pro Tabs #1/Banjo/Abernathy/Abernathy, Barry - You Still Call Me Baby.gp3', 'rb') as file:
    song = guitarpro.parse(file)
    track = guitarpro.Track(song)

print('Beat position expressed in 16th of measure')
for track in song.tracks:
    print('Track', track.number, track.name)
    for measure in track.measures:
        print('')
        print('----Measure', measure.number)
        for voice in measure.voices:
            for beat in voice.beats:
                print('-Beat start',int((beat.start/measure.length+1-measure.number)*16-4),'duration', int(1/beat.duration.value*16))
                for note in beat.notes:
                    print('String',note.string,'Fret', note.value)

Beat position expressed in 16th of measure
Track 1 Banjo

----Measure 1
-Beat start 0 duration 8
String 1 Fret 0
String 5 Fret 0
-Beat start 8 duration 4
String 4 Fret 0
-Beat start 12 duration 4
String 3 Fret 0

----Measure 2
-Beat start 0 duration 1
String 3 Fret 2
-Beat start 1 duration 1
String 3 Fret 3
-Beat start 2 duration 2
String 2 Fret 0
-Beat start 4 duration 2
String 1 Fret 0
-Beat start 6 duration 2
String 5 Fret 0
-Beat start 8 duration 2
String 1 Fret 0
-Beat start 10 duration 2
String 2 Fret 0
-Beat start 12 duration 1
String 3 Fret 3
-Beat start 13 duration 1
String 3 Fret 2
-Beat start 14 duration 2
String 1 Fret 0

----Measure 3
-Beat start 0 duration 4
String 3 Fret 0
-Beat start 4 duration 2
String 5 Fret 0
-Beat start 6 duration 2
String 1 Fret 0
-Beat start 8 duration 1
String 3 Fret 2
-Beat start 9 duration 1
String 3 Fret 3
-Beat start 10 duration 2
String 2 Fret 0
-Beat start 12 duration 2
String 1 Fret 0
-Beat start 14 duration 2
String 5 Fret 0

----Measure 

String 1 Fret 3
String 2 Fret 3
String 3 Fret 0

----Measure 10
-Beat start 0 duration 8
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
String 6 Fret 3
-Beat start 12 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0

----Measure 11
-Beat start 0 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
-Beat start 4 duration 8
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0
-Beat start 12 duration 4
String 1 Fret 3
String 2 Fret 3
String 3 Fret 0

----Measure 12
-Beat start 0 duration 4
String 2 Fret 3
String 3 Fret 2
String 4 Fret 0
-Beat start 4 duration 8
String 2 Fret 3
String 3 Fret 2
-Beat start 12 duration 4
String 2 Fret 3
String 3 Fret 2

----Measure 13
-Beat start 0 duration 4
String 2 Fret 3
String 3 Fret 2
-Beat start 4 duration 8
String 2 Fret 3
String 3 Fret 2
-Beat start 12 duration 4
String 2 Fret 3
String 3 Fret 2

----Measure 14
-Beat start 0 duration 4
String 2 Fret 3
String 3 Fret 2
-Beat start 4 duration 8
String 2 Fret 3
String 3 Fret 2
-Beat start 12 

In [14]:
NOTES = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']

A4_MIDI_VALUE = 69

class Note:
    """
    Represents a musical note 
    
    Parameters
    ---------------
    gpnote: guitarpro.models.Note (optional)
            If given, initialize the note to GuitarPro equivalent
            Must be given with tuning
    tuning: Array of guitarpro.models.GuitarString (optional)
            The tuning of the GuitarPro track from where the note comes from
    """
    def __init__(self, gpnote = None, tuning=None):
        self.letter = 'A'
        self.octave = 4
        self.freq = 440
        if gpnote:
            if not tuning:
                raise ValueError("gpnote should always be given with the instrument tuning")
            self._from_gpnote(gpnote, tuning)

        
    def _from_gpnote(self, gpnote, tuning):
        midi_value = None
        for s in tuning:
            if gpnote.string == s.number:
                midi_value = s.value + gpnote.value
                break
        
        if midi_value == None:
            raise ValueError("Error in converting GPnote: string %i, fret %i. Please check tuning is correct" % (gpnote.string,gpnote.value))
        
        self.octave = math.floor((midi_value - A4_MIDI_VALUE)/len(NOTES))+4
        self.letter = NOTES[(midi_value - A4_MIDI_VALUE) % len(NOTES)]
        
    def __str__(self):
        return self.letter + str(self.octave)
    
for track in song.tracks:
    print('****Track', track.number, track.name)
    for measure in track.measures:
        print('----Measure', measure.number)
        for voice in measure.voices:
            for beat in voice.beats:
                print('-Beat start',int((beat.start/measure.length+1-measure.number)*16-4),'duration', int(1/beat.duration.value*16))
                for note in beat.notes:
                    print(Note(note, track.strings), end=' ')
                print('')
        print('')
    print('')
    print('')

****Track 1 Banjo
----Measure 1
-Beat start 0 duration 8
D3 G3 
-Beat start 8 duration 4
D2 
-Beat start 12 duration 4
G2 

----Measure 2
-Beat start 0 duration 1
A3 
-Beat start 1 duration 1
A#3 
-Beat start 2 duration 2
B3 
-Beat start 4 duration 2
D3 
-Beat start 6 duration 2
G3 
-Beat start 8 duration 2
D3 
-Beat start 10 duration 2
B3 
-Beat start 12 duration 1
A#3 
-Beat start 13 duration 1
A3 
-Beat start 14 duration 2
D3 

----Measure 3
-Beat start 0 duration 4
G2 
-Beat start 4 duration 2
G3 
-Beat start 6 duration 2
D3 
-Beat start 8 duration 1
A3 
-Beat start 9 duration 1
A#3 
-Beat start 10 duration 2
B3 
-Beat start 12 duration 2
D3 
-Beat start 14 duration 2
G3 

----Measure 4
-Beat start 0 duration 1
C#3 
-Beat start 1 duration 1
D3 
-Beat start 2 duration 2
D3 
-Beat start 4 duration 2
G3 
-Beat start 6 duration 2
D3 
-Beat start 8 duration 2
D3 
-Beat start 10 duration 2
G3 
-Beat start 12 duration 1
C3 
-Beat start 13 duration 1
B3 
-Beat start 14 duration 2
D3 

----